In [1]:
import random
import numpy as np
from copy import copy, deepcopy
from adversarial_attack.pcp_utils import *

#### Create the HT model and load pretrained weights

In [3]:
import pickle
from dnn_model.CNN_netlist_softma_save_resluts import Classifier_Netlist

with open('save/source_config.pkl', 'rb') as pickle_file:
    source_config_copy = pickle.load(pickle_file)

path = './weights/CNN_model_pretrained.pth'
HTnn_net = Classifier_Netlist(group_id=str(2), base_path='json_temp_file', source_config=source_config_copy, pretrained=path)  
HTnn_net.evaluate_no_save() 

Chunking...0/3
Read 1M Feature Traces.
Chunking...1/3
Read 2M Feature Traces.
Chunking...2/3
Read 3M Feature Traces.
Chunking...Last
Read 4M Feature Traces.
Iterable Dataset Loaded...
===TEST===
Chunking...0/1
Chunking...Last
Read 1M Feature Traces.
Iterable Dataset Loaded...
Model loaded
2024-07-18 10:02:50,236 KFD[2]_nnclassifier INFO: ====TEST====


33716it [02:12, 254.24it/s]                         

2024-07-18 10:05:02,852 KFD[2]_nnclassifier INFO: Test Loss: 0.280729, Acc: 0.998739


2024-07-18 10:05:02,854 KFD[2]_nnclassifier INFO: tensor([[1.0767e+06, 2.9300e+02],
        [1.0400e+03, 8.4400e+02]])
2024-07-18 10:05:02,877 KFD[2]_nnclassifier INFO: tensor([0.9997, 0.4480])
2024-07-18 10:05:02,879 KFD[2]_nnclassifier INFO: tensor([0.9990, 0.7423])


### Get all the trojan circuits and choose one for adversarial modifications (e.g., 'troj33_0U4@c6288_T033_noassign_vallogic')

In [5]:
all_text_labels = get_all_text_labels(HTnn_net.val_dataloader)
trojan_comps_labels = []
for elem in all_text_labels:
    if elem.startswith("t"):
        trojan_comps_labels.append(elem)

trojan_comp = trojan_comps_labels[0] ## Choose one trojna circuit (automate later)
pcp_embs = get_samples_by_text_label(HTnn_net.val_dataloader, trojan_comp)
trojan_comps_labels

['troj33_0U4@c6288_T033_noassign_vallogic',
 'troj7_0U4@s15850_T007_noassign_vallogic',
 'trojan15_0@s13207_T015_noassign_vallogic',
 'troj64_0U4@c3540_T064_noassign_vallogic',
 'troj7_0U3@s15850_T007_noassign_vallogic',
 'troj2_0U3@c2670_T002_noassign_vallogic',
 'troj17_0U7@s35932_T017_noassign_vallogic',
 'trojan2_0@c2670_T002_noassign_vallogic',
 'troj17_0U3@s35932_T017_noassign_vallogic',
 'troj64_0U3@c3540_T064_noassign_vallogic',
 'troj17_0U5@s35932_T017_noassign_vallogic',
 'troj58_0U1@c5315_T058_noassign_vallogic',
 'troj10_0U3@s1423_T010_noassign_vallogic',
 'troj58_0U2@c5315_T058_noassign_vallogic',
 'troj15_0U1@s13207_T015_noassign_vallogic',
 'troj10_0U1@s1423_T010_noassign_vallogic',
 'troj17_0U6@s35932_T017_noassign_vallogic',
 'troj33_0U2@c6288_T033_noassign_vallogic',
 'trojan10_0@s1423_T010_noassign_vallogic',
 'troj33_0U1@c6288_T033_noassign_vallogic',
 'troj64_0U5@c3540_T064_noassign_vallogic',
 'troj15_0U3@s13207_T015_noassign_vallogic',
 'troj58_0U6@c5315_T058_noa

### Get all the PCP embeddings and component names for the pre-selected trojan circuit

In [7]:
all_embds, all_cmps, all_labels = [], [], []
for pcp_emb in pcp_embs:
    p_emb, label = pcp_emb
    full_pcp_cmp = []
    for i in range(5):
        name = get_cmp_by_emb(HTnn_net.val_data.word2vec_dict, list(np.float32(p_emb[i])))
        full_pcp_cmp.append(name)
    
    all_labels.append(label.item())
    all_embds.append(p_emb)
    all_cmps.append(full_pcp_cmp)

for cmp, label in zip(all_cmps, all_labels):
    print(cmp, label)

['DIN1_nor2_Q', 'DIN1_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] 1
['DIN1_nor2_Q', 'DIN1_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] 1
['DIN2_nor2_Q', 'DIN1_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] 1
['DIN2_nor2_Q', 'DIN1_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] 1
['DIN1_and2_Q', 'DIN2_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] 1
['DIN1_and2_Q', 'DIN2_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] 1
['DIN2_and2_Q', 'DIN2_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] 1
['DIN2_and2_Q', 'DIN2_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] 1
['DIN1_nor2_Q', 'DIN1_nor2_Q', 'DIN2_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] 1
['DIN1_nor2_Q', 'DIN1_nor2_Q', 'DIN2_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] 1
['DIN2_nor2_Q', 'DIN1_nor2_Q', 'DIN2_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] 1
['DIN2_nor2_Q', 'DIN1_nor2_Q', 'DIN2_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] 1
['DIN1_nor2_Q', 'DIN2_nor2_Q', 'DIN2_nnd4_Q', 'DIN3_nor5_Q', 'DI

### Define a genetic search algorithm to find an approximate HT circuit with optimal tradeoff between approx-error and HT-detect-score

In [11]:
def genetic_search(HTnn_net, orig_pcp_list, population_size, generations):
    population = [mutate_pcp_list(HTnn_net, deepcopy(orig_pcp_list), n_changes=random.randint(1, 2)) for _ in range(population_size)]  
    
    best_solutions = []
    
    for _ in range(generations):
        fitness_scores, appx_errs, dect_errs = [], [], []
        for i in range(len(population)):
            appx_err = approximation_error(deepcopy(orig_pcp_list), deepcopy(population[i]))
            dect_err = detect_score(HTnn_net, get_all_embeddings(HTnn_net, deepcopy(population[i])))
            appx_errs.append(round(appx_err, 2))
            dect_errs.append(round(dect_err, 2))
            fitness_scores.append(appx_err+dect_err)
                
        # Update best solutions
        for i, fitness in enumerate(fitness_scores):
            best_solutions.append((deepcopy(population[i]), fitness, appx_errs[i], dect_errs[i]))
        best_solutions.sort(key=lambda x: x[1])  # Sort by fitness score
        best_solutions = best_solutions[:1]  # Keep only top 10
        
        parents = random.choices(population, weights=fitness_scores, k=population_size)   
        new_population = []
        for i in range(0, population_size, 2):
            parent1, parent2 = parents[i], parents[i+1]    
            child1 = mutate_pcp_list(HTnn_net, deepcopy(parent1))
            child2 = mutate_pcp_list(HTnn_net, deepcopy(parent2))
            new_population.extend([child1, child2])
        
        population = new_population
    
    return best_solutions

### Run the genetic search on the pre-selected trojan circuit (i.e., 'troj33_0U4@c6288_T033_noassign_vallogic')

In [16]:
best_solution = genetic_search(HTnn_net, deepcopy(all_cmps), population_size=128, generations=10)
print('********* Average Approximation Error', best_solution[0][2])
print('********* HT Detection Error', best_solution[0][3])
print('********* Show changes in the PCP traces (Original PCP --> New PCP)')
for old_elem, new_elem in zip(deepcopy(all_cmps), best_solution[0][0]):
    print('{} ----> {}'.format(old_elem, new_elem))


********* Average Approximation Error 0.05
********* HT Detection Error 0.0
********* Show changes in the PCP traces (Original PCP --> New PCP)
['DIN1_nor2_Q', 'DIN1_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] ----> ['DIN1_nor2_Q', 'DIN1_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_nnd2_Q']
['DIN1_nor2_Q', 'DIN1_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] ----> ['DIN1_nor2_Q', 'DIN1_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_nnd2_Q']
['DIN2_nor2_Q', 'DIN1_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] ----> ['DIN2_nor2_Q', 'DIN1_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_nnd2_Q']
['DIN2_nor2_Q', 'DIN1_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] ----> ['DIN2_nor2_Q', 'DIN1_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_nnd2_Q']
['DIN1_and2_Q', 'DIN2_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_xor2_Q'] ----> ['DIN1_and2_Q', 'DIN2_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2_nnd2_Q']
['DIN1_and2_Q', 'DIN2_nor2_Q', 'DIN1_nnd4_Q', 'DIN3_nor5_Q', 'DIN2